<br>
<img src="https://i.ibb.co/7j6FB3h/8x8-logo.png" width="240" height="50" align="center"><br>


## Transaction

### Next db update should include data from: 2021-06-19

In [67]:
# Read necessary libraries
import pandas as pd                                             # data manipulation and analysis
import numpy as np                                              # data manipulation and analysis
pd.set_option('display.max_columns', None)                      # show all columns
from datetime import datetime                                   # classes for manipulating dates and times
import warnings                                                 # indicating warnings
warnings.filterwarnings('ignore')                               # ignore by default
pd.set_option('display.max_rows', None)                         # show all rows
import pymysql                                                  # MySql connection

In [42]:
# read main file as 'df'
# Choose required columns only
column_names = ['queue', 'transaction', 'phone', 'create_time', 'agent', 'accept_time', 'processing_time',
                'post_processing_time','processing_and_post_time','abandon_time','hold_max','hold_count','hold_total']

# Read excel file
df = pd.read_excel(r'C:\Users\I-SURANCE\Desktop\BM Sprint\8x8\Part1\Files\detailed_entered_transactions_activity.xlsx'
                         ,skiprows= [0,1,2,3,4,5,6,7], usecols = [2,3,4,8,9,10,11,12,13,14,17,18,19],names = column_names)

# convert to secs
df['processing_time'] = pd.to_timedelta(df['processing_time'].astype(str)).dt.seconds
df['post_processing_time'] = pd.to_timedelta(df['post_processing_time'].astype(str)).dt.seconds
df['processing_and_post_time'] = pd.to_timedelta(df['processing_and_post_time'].astype(str)).dt.seconds
df['hold_max'] = pd.to_timedelta(df['hold_max'].astype(str)).dt.seconds
df['hold_total'] = pd.to_timedelta(df['hold_total'].astype(str)).dt.seconds

# convert object to date type
df['create_time'] =  pd.to_datetime(df['create_time'])
df['accept_time'] =  pd.to_datetime(df['accept_time'])
df['abandon_time'] =  pd.to_datetime(df['abandon_time'])

                        ###################################################################
                        ############# extract partner and language from queue #############
                        ###################################################################
# split queue
language_partner = df['queue'].str.split(r"-|_|\s+", expand=True)
# Manage Queues' name here
language_partner = language_partner[language_partner.columns].apply(lambda x: x.replace({'ID': np.nan, 'fnac': 'FNAC', 
    'Fnac': 'FNAC', 'AudikaCH': 'Audika', 'DigitecGLISE':'Digitec', 'UPC':np.nan, 'Paris':np.nan, 'Smart':np.nan, 
    'IDProtect':np.nan,'Zendesk':np.nan, 'DisplayProtect':np.nan, 'Micare':np.nan, 'DisplayprotectES':np.nan, 
    'DisplayprotectES':np.nan,' ':np.nan, 'customers':np.nan, 'complaints':np.nan, 'GLISE':np.nan, 'claims':np.nan, 
    'cancellations':np.nan,'DE':'German', 'FR':'French', 'CH':np.nan, 'Telecom':np.nan, 'EN':'English', 'IT':'Italian', 
    'ES':'Spanish','store':np.nan, 'office':np.nan, 'Talk':np.nan, 'Others':np.nan, 'Complaints':np.nan, 'complaint':np.nan,
    'contract':np.nan, 'T':np.nan, 'cbq':np.nan, 'fr':'French', 'de':'German', 'it':'Italian', 'en':'English',
    'pt':'Portuguese', 'team':np.nan, 'Queue':np.nan, 'information':np.nan, 'opq':np.nan, 'notification':np.nan,
    'salt':'Salt', 'Monaco':'MonacoTelecom', 'Displayprotect':np.nan, 'Protect':np.nan, 'claim':np.nan, 'customer':np.nan,
    'Test':np.nan, 'Customers':np.nan, 'es':'Spanish', 'Customer':np.nan, 'PT':'Portuguese', 'PL':'Polish', None:np.nan,
    'Displayprotect':np.nan, 'yallo':'Yallo', '  ':np.nan, 'Claims':np.nan}))
# Deleting white spaces
language_partner = language_partner.replace(r'^\s*$', np.nan, regex=True)
# make 2 desirable columns out of 7
language_partner = language_partner.fillna('')
language_partner['Language2'] = language_partner[1]+language_partner[2]+language_partner[3]+language_partner[4]+\
                            language_partner[5]+language_partner[6]
language_partner.drop(columns = [1,2,3,4,5,6,7], inplace = True)
language_partner.columns = ['partner','language']
# Merge based on indexes
df = df.merge(language_partner, left_index=True, right_index=True)
# Replace empty values '' with np.nan
df.at[df.loc[(df['language'] == '')].index,'language'] = np.nan
df.at[df.loc[(df['partner'] == '')].index,'partner'] = np.nan

                        ###################################################################
    
# extract 3 more columns based on 'queue'
df['complaint'] = df['queue'].apply(lambda string: 'Yes' if 'complaint' in string else 'No')
df['outbound'] = df['queue'].apply(lambda string: 'Yes' if 'opq' in string else 'No')
df['callback'] = df['queue'].apply(lambda string: 'Yes' if 'cbq' in string else 'No')

# drop 'queue'
df.drop(columns=['queue'], inplace = True)

# organize 'agent'
df['agent'] = df['agent'].str.replace(r" \(.*\)","")

# drop test calls
df.drop(df.loc[df['agent'].isin(['ISurance ZendeskTest', 'Nikola Test'])].index, inplace = True)

# replace wrong phone numbers
wrong_numbers = df['phone'].str.contains('^[a-zA-Z]$')
wrong_numbers = wrong_numbers[wrong_numbers.notnull()].index
df.at[wrong_numbers,'phone'] = np.nan
df['phone'] = df['phone'].astype(float)

# tell numbers whose length are less than 7 or more than 13 digits cannot be identified/correct
wrong_tell_length = df.loc[(df['phone'] < 1000000) | (df['phone'] > 1000000000000)].index
df.at[wrong_tell_length,'phone'] = np.nan

# Country codes
df['country'] = np.nan
df.at[df.loc[df['phone'].astype(str).str.match('33')].index,'country'] = 'France'
df.at[df.loc[df['phone'].astype(str).str.match('351')].index,'country'] = 'Portugal'
df.at[df.loc[df['phone'].astype(str).str.match('32')].index,'country'] = 'Belgium'
df.at[df.loc[df['phone'].astype(str).str.match('31')].index,'country'] = 'Netherlands'
df.at[df.loc[df['phone'].astype(str).str.match('39')].index,'country'] = 'Italy'
df.at[df.loc[df['phone'].astype(str).str.match('41')].index,'country'] = 'Switzerland'
df.at[df.loc[df['phone'].astype(str).str.match('48')].index,'country'] = 'Poland'
df.at[df.loc[df['phone'].astype(str).str.match('44')].index,'country'] = 'UK'
df.at[df.loc[df['phone'].astype(str).str.match('43')].index,'country'] = 'Austria'
df.at[df.loc[df['phone'].astype(str).str.match('34')].index,'country'] = 'Spain'
df.at[df.loc[df['phone'].astype(str).str.match('353')].index,'country'] = 'Ireland'
df.at[df.loc[df['phone'].astype(str).str.match('49')].index,'country'] = 'Germany'
df.at[df.loc[df['phone'].astype(str).str.match('423')].index,'country'] = 'Liechtenstein'
df.at[df.loc[df['phone'].astype(str).str.match('46')].index,'country'] = 'Sweden'
df.at[df.loc[df['phone'].astype(str).str.match('377')].index,'country'] = 'Monaco'
df.at[df.loc[df['phone'].astype(str).str.match('45')].index,'country'] = 'Denmark'
df.at[df.loc[df['phone'].astype(str).str.match('352')].index,'country'] = 'Luxembourg'
df.at[df.loc[df['phone'].astype(str).str.match('385')].index,'country'] = 'Croatia'
df.at[df.loc[df['phone'].astype(str).str.match('372')].index,'country'] = 'Estonia'
df.at[df.loc[df['phone'].astype(str).str.match('40')].index,'country'] = 'Romania'
df.at[df.loc[df['phone'].astype(str).str.match('380')].index,'country'] = 'Ukraine'
df.at[df.loc[df['phone'].astype(str).str.match('36')].index,'country'] = 'Hungary'

# defining necessary metrics
df['entered'] = 1

df['accepted'] = 1
df.at[df.loc[df['accept_time'].isnull()].index,'accepted'] = 0

df['abandoned'] = 0
df.at[df.loc[df['abandon_time'].notnull()].index,'abandoned'] = 1

df['accept_create_diff_time'] = (df['accept_time'] - df['create_time']).dt.seconds
df['SLA'] = np.nan
df.at[df.loc[df['accept_create_diff_time'] <= 60].index ,'SLA'] = 1
df.at[df.loc[df['accept_create_diff_time'] > 60].index ,'SLA'] = 0

df['abandon_create_diff_time'] = (df['abandon_time'] - df['create_time']).dt.seconds

df['abandon_5s'] = np.nan
df['abandon_6s_10s'] = np.nan
df['abandon_11s_20s'] = np.nan
df['abandon_21s_30s'] = np.nan
df['abandon_31s_45s'] = np.nan
df['abandon_46s_60s'] = np.nan
df['abandon_61s_120s'] = np.nan
df['abandon_121s_300s'] = np.nan
df['abandon_301s_600s'] = np.nan
df['abandon_601s'] = np.nan
df.at[df.loc[df['abandoned'] == 1].index,'abandon_5s'] = 0
df.at[df.loc[df['abandoned'] == 1].index,'abandon_6s_10s'] = 0
df.at[df.loc[df['abandoned'] == 1].index,'abandon_11s_20s'] = 0
df.at[df.loc[df['abandoned'] == 1].index,'abandon_21s_30s'] = 0
df.at[df.loc[df['abandoned'] == 1].index,'abandon_31s_45s'] = 0
df.at[df.loc[df['abandoned'] == 1].index,'abandon_46s_60s'] = 0
df.at[df.loc[df['abandoned'] == 1].index,'abandon_61s_120s'] = 0
df.at[df.loc[df['abandoned'] == 1].index,'abandon_121s_300s'] = 0
df.at[df.loc[df['abandoned'] == 1].index,'abandon_301s_600s'] = 0
df.at[df.loc[df['abandoned'] == 1].index,'abandon_601s'] = 0
df.at[df.loc[df['abandon_create_diff_time'].between(0,5)].index,'abandon_5s'] = 1
df.at[df.loc[df['abandon_create_diff_time'].between(6,10)].index,'abandon_6s_10s'] = 1
df.at[df.loc[df['abandon_create_diff_time'].between(11,20)].index,'abandon_11s_20s'] = 1
df.at[df.loc[df['abandon_create_diff_time'].between(21,30)].index,'abandon_21s_30s'] = 1
df.at[df.loc[df['abandon_create_diff_time'].between(31,45)].index,'abandon_31s_45s'] = 1
df.at[df.loc[df['abandon_create_diff_time'].between(46,60)].index,'abandon_46s_60s'] = 1
df.at[df.loc[df['abandon_create_diff_time'].between(61,120)].index,'abandon_61s_120s'] = 1
df.at[df.loc[df['abandon_create_diff_time'].between(121,300)].index,'abandon_121s_300s'] = 1
df.at[df.loc[df['abandon_create_diff_time'].between(301,600)].index,'abandon_301s_600s'] = 1
df.at[df.loc[df['abandon_create_diff_time'] >= 601].index,'abandon_601s'] = 1

# Order by created time
df = df.sort_values(['create_time'], ascending=[False])
df = df.reset_index(drop=True)

# merge with wrap-up file
column_names = ['transaction', 'wrapup', 'accept_time']
wrapup = pd.read_excel(
    r'C:\Users\I-SURANCE\Desktop\BM Sprint\8x8\Part1\Files\detailed_accepted_transactions_with_wrap_up_codes.xlsx'
                         ,skiprows= [0,1,2,3,4,5,6,7], usecols = [4,6,9],names = column_names)
wrapup = wrapup[wrapup['transaction'].notna()]
wrapup['accept_time'] =  pd.to_datetime(wrapup['accept_time'])
df = df.merge(wrapup, on=['transaction','accept_time'], how='left')

# there are 8 identical rows as a result of merge
df = df.drop_duplicates(subset=['transaction', 'phone', 'create_time','accept_time', 'agent'], keep = 'first')

# 'phone' makes '.0' in db. This is the solution
df['phone'] = df['phone'].astype(str).apply(lambda x: x.replace('.0',''))
df.at[df.loc[df['phone'] == 'nan'].index,'phone'] = np.nan


In [57]:
df.head(138)

,transaction,phone,create_time,agent,accept_time,processing_time,post_processing_time,processing_and_post_time,abandon_time,hold_max,hold_count,hold_total,partner,language,complaint,outbound,callback,country,entered,accepted,abandoned,accept_create_diff_time,SLA,abandon_create_diff_time,abandon_5s,abandon_6s_10s,abandon_11s_20s,abandon_21s_30s,abandon_31s_45s,abandon_46s_60s,abandon_61s_120s,abandon_121s_300s,abandon_301s_600s,abandon_601s,wrapup
0,308935,41786967227,2021-06-18 18:51:05,Anastazia Dupuy Moric,2021-06-18 18:51:10,196.0,131.0,327.0,NaT,0.0,0.0,0.0,Salt,French,No,No,No,Switzerland,1,1,0,5.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FMIP
1,308773,41787877255,2021-06-18 18:24:12,Anastazia Dupuy Moric,2021-06-18 18:24:20,64.0,141.0,205.0,NaT,0.0,0.0,0.0,Salt,French,No,No,No,Switzerland,1,1,0,8.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Partner Calls
2,308748,41786162526,2021-06-18 18:21:59,Gabrijela Nikolic,2021-06-18 18:22:09,660.0,247.0,907.0,NaT,99.0,1.0,99.0,Salt,German,No,No,No,Switzerland,1,1,0,10.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complaint
3,308500,41787877255,2021-06-18 17:56:37,Mouna Mbarki,2021-06-18 17:56:51,347.0,10.0,357.0,NaT,0.0,0.0,0.0,Salt,French,No,No,No,Switzerland,1,1,0,14.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Claim Online
4,308430,49306052363,2021-06-18 17:52:46,NaN,NaT,NaN,NaN,NaN,2021-06-18 17:58:10,0.0,0.0,0.0,Audibene,German,No,No,No,Germany,1,0,1,NaN,NaN,324.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
5,308399,41443800422,2021-06-18 17:51:32,Anastasio Gonzalez,2021-06-18 17:52:01,342.0,375.0,717.0,NaT,0.0,0.0,0.0,Salt,German,No,No,No,Switzerland,1,1,0,29.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,General Information and Documents
6,308384,41763412108,2021-06-18 17:50:46,Silvija Sabo,2021-06-18 17:50:48,148.0,382.0,530.0,NaT,0.0,0.0,0.0,Salt,German,No,No,No,Switzerland,1,1,0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
7,308375,41786967886,2021-06-18 17:50:42,Gabrijela Nikolic,2021-06-18 17:50:55,417.0,45.0,462.0,NaT,0.0,0.0,0.0,Salt,German,No,No,No,Switzerland,1,1,0,13.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Warranty on Repair
8,308334,49306052363,2021-06-18 17:48:14,NaN,NaT,NaN,NaN,NaN,2021-06-18 17:51:15,0.0,0.0,0.0,Audibene,German,No,No,No,Germany,1,0,1,NaN,NaN,181.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN
9,308273,33650747136,2021-06-18 17:45:06,Mouna Mbarki,2021-06-18 17:45:28,383.0,201.0,584.0,NaT,0.0,0.0,0.0,BackMarket,French,No,No,No,France,1,1,0,22.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Claim process


In [58]:
# save the file
today = datetime.now().strftime("%Y-%m-%d")
name = '8x8_transaction_' + str(today) + '.csv'
name = name.replace('/', '_')
df.to_csv(name, sep=',', encoding='utf-8-sig', index = False)

## Agent

In [85]:
# read main file as 'df'
# Choose required columns only
column_names = ['period','agent','waiting','processing_time','post_processing_time', 'working_offline', 'transaction_offered',
                'break', 'processing_and_post_time', 'logged_in']

# Read excel file
agent_time = pd.read_excel(r'C:\Users\I-SURANCE\Desktop\BM Sprint\8x8\Part1\Files\time_on_status.xlsx'
                         ,skiprows= [0,1,2,3,4,5,6,7], usecols = [0,1,2,4,6,8,10,12,14,16],names = column_names)

# read main file as 'df'
# Choose required columns only
column_names = ['period','agent','accepted', 'rejected', 'presented']

# Read excel file
agent_call = pd.read_excel(r'C:\Users\I-SURANCE\Desktop\BM Sprint\8x8\Part1\Files\transactions.xlsx'
                         ,skiprows= [0,1,2,3,4,5,6,7], usecols = [0,1,2,3,4],names = column_names)

############################################################

agent_time['waiting'] = agent_time['waiting'].astype(str).apply(lambda x: x.replace('1900-01-01',''))
agent_time['processing_time'] = agent_time['processing_time'].astype(str).apply(lambda x: x.replace('1900-01-01',''))
agent_time['post_processing_time'] = agent_time['post_processing_time'].astype(str).apply(lambda x: x.replace('1900-01-01',''))
agent_time['working_offline'] = agent_time['working_offline'].astype(str).apply(lambda x: x.replace('1900-01-01',''))
agent_time['transaction_offered'] = agent_time['transaction_offered'].astype(str).apply(lambda x: x.replace('1900-01-01',''))
agent_time['break'] = agent_time['break'].astype(str).apply(lambda x: x.replace('1900-01-01',''))
agent_time['processing_and_post_time'] = agent_time['processing_and_post_time'].astype(str).apply(lambda x: x.replace('1900-01-01',''))
agent_time['logged_in'] = agent_time['logged_in'].astype(str).apply(lambda x: x.replace('1900-01-01',''))

agent_time['waiting'] = pd.to_timedelta(agent_time['waiting'].astype(str)).dt.seconds
agent_time['processing_time'] = pd.to_timedelta(agent_time['processing_time'].astype(str)).dt.seconds
agent_time['post_processing_time'] = pd.to_timedelta(agent_time['post_processing_time'].astype(str)).dt.seconds
agent_time['working_offline'] = pd.to_timedelta(agent_time['working_offline'].astype(str)).dt.seconds
agent_time['transaction_offered'] = pd.to_timedelta(agent_time['transaction_offered'].astype(str)).dt.seconds
agent_time['break'] = pd.to_timedelta(agent_time['break'].astype(str)).dt.seconds
agent_time['processing_and_post_time'] = pd.to_timedelta(agent_time['processing_and_post_time'].astype(str)).dt.seconds
agent_time['logged_in'] = pd.to_timedelta(agent_time['logged_in'].astype(str)).dt.seconds

agent_time['period'] =  pd.to_datetime(agent_time['period'])

agent_time['agent'] = agent_time['agent'].str.replace(r" \(.*\)","")

agent_time.drop(agent_time.loc[agent_time['agent'].isin(['ISurance ZendeskTest'])].index, inplace = True)

############################################################

agent_call['period'] =  pd.to_datetime(agent_call['period'])

agent_call['agent'] = agent_call['agent'].str.replace(r" \(.*\)","")

agent_call.drop(agent_call.loc[agent_call['agent'].isin(['ISurance ZendeskTest'])].index, inplace = True)

###########################################################

agent = agent_time.merge(agent_call, on=['period','agent'], how='left')

# Order by created time
agent = agent.sort_values(['period'], ascending=[False])
agent = agent.reset_index(drop=True)


In [86]:
agent.head(100)

,period,agent,waiting,processing_time,post_processing_time,working_offline,transaction_offered,break,processing_and_post_time,logged_in,accepted,rejected,presented
0,2021-06-18,Aimar Vellve,3020,0,0,22370,0,3626,0,29016,NaN,NaN,NaN
1,2021-06-18,Anastasio Gonzalez,1954,1666,591,23959,17,6463,2257,34650,NaN,NaN,NaN
2,2021-06-18,Paloma Bosco,25502,0,0,0,0,10,0,25512,NaN,NaN,NaN
3,2021-06-18,Mouna Mbarki,13289,6119,1993,3805,164,3568,8112,28938,13.0,0.0,13.0
4,2021-06-18,Meryem Bijja,16281,6157,796,2332,90,3180,6953,28836,14.0,0.0,14.0
5,2021-06-18,Maud Duterme,4226,975,111,19110,14,4676,1086,29112,2.0,0.0,2.0
6,2021-06-18,Marion Loizeau,3906,0,0,17583,0,5,0,21494,NaN,NaN,NaN
7,2021-06-18,Mariano Atan,16298,4830,715,4332,125,5952,5545,32252,14.0,0.0,14.0
8,2021-06-18,Ludivine Germany,14418,0,0,4,0,7,0,14429,NaN,NaN,NaN
9,2021-06-18,Janine Dulken,15749,5567,1678,3006,279,2635,7245,28914,17.0,2.0,19.0


In [87]:
# save the file
today = datetime.now().strftime("%Y-%m-%d")
name = '8x8_agent_' + str(today) + '.csv'
name = name.replace('/', '_')
agent.to_csv(name, sep=',', encoding='utf-8-sig', index = False)

## Loading into microservices

In [ ]:
# Transaction (df)
# For more information look at: https://www.dataquest.io/blog/sql-insert-tutorial/
conn = pymysql.connect(host='microservices-bi.cdks3icyripq.eu-central-1.rds.amazonaws.com', port=3306, user='micro_bi', 
                       password='zoo7ooshikifoox', db='microservices_bi')

cursor=conn.cursor()

# np.nan cannot be executed
df = df.replace({np.nan: None})

# creating column list for insertion
cols = "`,`".join([str(i) for i in df.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df.iterrows():
    sql = "INSERT INTO `8x8_transaction` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    conn.commit()

In [83]:
# Agent (agent)
# For more information look at: https://www.dataquest.io/blog/sql-insert-tutorial/
conn = pymysql.connect(host='microservices-bi.cdks3icyripq.eu-central-1.rds.amazonaws.com', port=3306, user='micro_bi', 
                       password='zoo7ooshikifoox', db='microservices_bi')

cursor=conn.cursor()

# np.nan cannot be executed
agent = agent.replace({np.nan: None})

# creating column list for insertion
cols = "`,`".join([str(i) for i in agent.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in agent.iterrows():
    sql = "INSERT INTO `8x8_agent` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    conn.commit()